In [1]:
from enum import Enum

In [2]:
Activation = Enum('Activation', 'ReLU Swish GeLU')

In [100]:
feed_forward??

In [3]:
import torch
from pytorch_pretrained_bert import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel

In [4]:
import torch.nn as nn

In [105]:
from torch.nn.modules.activation import ReLU

In [102]:
from fastai.text.models import MultiHeadRelativeAttention,DecoderLayer,MultiHeadAttention,feed_forward

In [56]:
MultiHeadRelativeAttention??

In [55]:
MultiHeadAttention??

In [36]:
model = TransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103')
model.eval()

TransfoXLLMHeadModel(
  (transformer): TransfoXLModel(
    (word_emb): AdaptiveEmbedding(
      (emb_layers): ModuleList(
        (0): Embedding(20000, 1024)
        (1): Embedding(20000, 256)
        (2): Embedding(160000, 64)
        (3): Embedding(67735, 16)
      )
      (emb_projs): ParameterList(
          (0): Parameter containing: [torch.FloatTensor of size 1024x1024]
          (1): Parameter containing: [torch.FloatTensor of size 1024x256]
          (2): Parameter containing: [torch.FloatTensor of size 1024x64]
          (3): Parameter containing: [torch.FloatTensor of size 1024x16]
      )
    )
    (drop): Dropout(p=0.1)
    (layers): ModuleList(
      (0): RelPartialLearnableDecoderLayer(
        (dec_attn): RelPartialLearnableMultiHeadAttn(
          (qkv_net): Linear(in_features=1024, out_features=3072, bias=False)
          (drop): Dropout(p=0.1)
          (dropatt): Dropout(p=0.0)
          (o_net): Linear(in_features=1024, out_features=1024, bias=False)
          (laye

In [37]:
# Load pre-trained model tokenizer (vocabulary from wikitext 103)
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')

# Tokenized input
text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"
tokenized_text_1 = tokenizer.tokenize(text_1)
tokenized_text_2 = tokenizer.tokenize(text_2)

# Convert token to vocabulary indices
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)

# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])

In [39]:
# If you have a GPU, put everything on cuda
tokens_tensor_1 = tokens_tensor_1.to('cuda')
tokens_tensor_2 = tokens_tensor_2.to('cuda')
model.to('cuda')

with torch.no_grad():
    # Predict all tokens
    predictions_1, mems_1 = model(tokens_tensor_1)
    # We can re-use the memory cells in a subsequent call to attend a longer context
    predictions_2, mems_2 = model(tokens_tensor_2, mems=mems_1)

# get the predicted last token
predicted_index = list(map(lambda x:x.item(),predictions_2[0, :, :].argmax(1)))
predicted_token = tokenizer.convert_ids_to_tokens(predicted_index)


In [40]:
with torch.no_grad():
    hidden_2, mems_2 = model.transformer(tokens_tensor_2, mems=mems_1)

In [41]:
hidden_2

tensor([[[ 0.1199,  0.1363,  0.1029,  ..., -0.3195, -0.3179, -0.0682],
         [ 0.2988,  0.0126, -0.2789,  ..., -0.3445, -0.0766,  0.0204],
         [ 0.0339,  0.0836, -0.2165,  ..., -0.3910, -0.1606,  0.0165],
         [ 0.1092,  0.2094, -0.1813,  ..., -0.4091, -0.1013, -0.0221],
         [-0.1104,  0.0851, -0.0909,  ..., -0.0560, -0.0288,  0.0116]]],
       device='cuda:0')

In [31]:
len(tokenizer.counter.items())

267734

In [35]:
TransfoXLModel??

In [43]:
model.children

<bound method Module.children of TransfoXLLMHeadModel(
  (transformer): TransfoXLModel(
    (word_emb): AdaptiveEmbedding(
      (emb_layers): ModuleList(
        (0): Embedding(20000, 1024)
        (1): Embedding(20000, 256)
        (2): Embedding(160000, 64)
        (3): Embedding(67735, 16)
      )
      (emb_projs): ParameterList(
          (0): Parameter containing: [torch.cuda.FloatTensor of size 1024x1024 (GPU 0)]
          (1): Parameter containing: [torch.cuda.FloatTensor of size 1024x256 (GPU 0)]
          (2): Parameter containing: [torch.cuda.FloatTensor of size 1024x64 (GPU 0)]
          (3): Parameter containing: [torch.cuda.FloatTensor of size 1024x16 (GPU 0)]
      )
    )
    (drop): Dropout(p=0.1)
    (layers): ModuleList(
      (0): RelPartialLearnableDecoderLayer(
        (dec_attn): RelPartialLearnableMultiHeadAttn(
          (qkv_net): Linear(in_features=1024, out_features=3072, bias=False)
          (drop): Dropout(p=0.1)
          (dropatt): Dropout(p=0.0)
     

In [108]:
default_wrd_conf = {
    "d_model":1024,
    "embed_p":0.,
    "n_heads":5,
    "d_head":64,
    "resid_p":0.0,
    "attn_p":0.0,
    "bias": False,
    "scale":True,
    "intermediate_size":3072,
    "ff_p":0.
}

In [109]:
default_ent_conf = default_wrd_conf.copy()

In [110]:
default_ent_conf["d_model"] = 100

In [111]:
mhra_keys = ["d_model","n_heads","d_head","resid_p","attn_p","bias","scale"]

In [112]:
{ k: default_ent_conf[k] for k in ["d_model","n_heads","d_head","resid_p","attn_p","bias","scale"] }

{'d_model': 100,
 'n_heads': 5,
 'd_head': 64,
 'resid_p': 0.0,
 'attn_p': 0.0,
 'bias': False,
 'scale': True}

In [117]:
class EntEmbedingLayer(torch.nn.Module):
    def __init__(vocab_sz_ent:int,ctx_len:int,d_model_ent:int):
        super().__init__()
        self.ent_enc = nn.Embedding(vocab_sz_ent, d_model_ent)

    def forword(ent_ids_tensor):
        return self.ent_enc(ent_ids_tensor)

class KETFeedForward(torch.nn.Module):
    def __init__(self, wrd_conf,ent_conf):
        super().__init__()
        self.dense = nn.Linear(wrd_conf["intermediate_size"], wrd_conf["d_model"])
        self.dense_ent = nn.Linear(wrd_conf["intermediate_size"], ent_conf["d_model"])
        self.LayerNorm = nn.LayerNorm(wrd_conf["d_model"], eps=1e-12)
        self.LayerNorm_ent = nn.LayerNorm(ent_conf["d_model"], eps=1e-12)
        self.dropout = nn.Dropout(wrd_conf["ff_p"])

    def forward(self, hidden_states_, input_tensor, input_tensor_ent):
        hidden_states = self.dense(hidden_states_)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)

        hidden_states_ent = self.dense_ent(hidden_states_)
        hidden_states_ent = self.dropout(hidden_states_ent)
        hidden_states_ent = self.LayerNorm_ent(hidden_states_ent + input_tensor_ent)
        return hidden_states, hidden_states_ent

class MaskedKnowledgeAGGBlock(torch.nn.Module):
    def __init__(self,wrd_conf,ent_conf):
        super().__init__()
        self.mhra_wrd = MultiHeadRelativeAttention(**{ k: wrd_conf[k] for k in mhra_keys })
        self.mhra_ent = MultiHeadRelativeAttention(**{ k: ent_conf[k] for k in mhra_keys })
        self.dense = nn.Linear(wrd_conf["d_model"], wrd_conf["intermediate_size"])
        self.dense_ent = nn.Linear(ent_conf["d_model"], wrd_conf["intermediate_size"])
        self.act = ReLU()
        self.ff = KETFeedForward(wrd_conf,ent_conf)

    def forward(self,hidden,hidden_ent,mask,mask_ent,r, u, v,r_ent,u_ent,v_ent, mem,mem_ent):
        hidden = self.mhra_wrd(hidden,r=r, u=u, v=v,mask=mask,mem=mem)
        hidden_ent = self.mhra_ent(hidden_ent,r=r_ent, u=u_ent, v=v_ent,mask=mask_ent,mem=mem_ent)
        # get knowledge and words to same dim
        intermediate = self.dense(hidden)+self.dense_ent(hidden_ent)
        intermediate = self.act(intermediate_size)
        # mixes the knowledge with the tokens
        hidden,hidden_ent = self.ff(intermediate,hidden,hidden_ent)
    

def create_MKAGG_submodules(ctx_len,conf):
    pos_enc = nn.Embedding(ctx_len, conf["d_model"])
    drop_emb = nn.Dropout(conf["embed_p"])
    u = nn.Parameter(torch.Tensor(conf["n_heads"], 1, conf["d_head"]))
    v = nn.Parameter(torch.Tensor(conf["n_heads"], 1, conf["d_head"]))
    return pos_enc,drop_emb,u,v

class MaskedKnowledgeAGG(torch.nn.Module):
    def __init__(self,
                 wrd_conf,
                    #  d_model:int,
                    #  d_head:int,
                    #  n_heads:int,
                    #  d_inner: int,
                    #  embed_p: float=0.,
                    #  resid_p: float=0.,
                    #  attn_p: float=0.,
                    #  ff_p: float=0.,
  
                 ent_conf,
                    #  d_model_ent: int,
                    #  d_head_ent: int,
                    #  n_heads_ent: int,
                    #  d_inner_ent: int,
                    #  embed_p_ent: float=0.,
                    #  resid_p_ent: float=0.,
                    #  attn_p_ent: float=0.,
                    #  ff_p_ent: float=0.,
                 
                 ctx_len: int,
                 mem_len:int,
                 n_layers:int,

                 mask:bool=True,

                ):
        super().__init__()
        # embeding should be done in before this module layers
        # self.token_enc = nn.Embedding(vocab_sz, d_model)
        self.pos_enc_wrd, self.drop_emb_wrd, self.u_wrd, self.v_wrd = create_MKAGG_submodules(ctx_len,wrd_conf)
        self.pos_enc_ent, self.drop_emb_ent, self.u_ent, self.v_ent = create_MKAGG_submodules(ctx_len,ent_conf)
        self.mem_len,self.n_layers,self.mask = mem_len,n_layers,mask
        
        self.layers = []
        for i in range(n_layers):
            self.layers.append(MaskedKnowledgeAGGBlock(wrd_conf,ent_conf))
            
        self.layers =  nn.ModuleList(self.layers)



    def _update_mems(self, mem,hids):
        if mem is None:
             return None
        assert len(hids) == len(mem), 'len(hids) != len(self.hidden)'
        with torch.no_grad():
            for i in range(len(hids)):
                cat = torch.cat([mem[1], hids[i]], dim=1)
            return  cat[:,-self.mem_len:].detach()

    def forward(self, x,x_ent,mem,mem_ent):
        #The hidden state has to be initiliazed in the forward pass for nn.DataParallel
        bs,x_len = x.size()
        inp = self.drop_emb_wrd(x) #.mul_(self.d_model ** 0.5)
        inp_ent = self.drop_emb_ent(x_ent)

        m_len = mem[0].size(1) if len(self.mem[0].size()) > 1 else 0
        
        seq_len = m_len + x_len
        
        mask = torch.triu(x.new_ones(x_len, seq_len), diagonal=1+m_len).byte()[None,None] if self.mask else None
         # need to addapt this map acording to traning task (
         #  for next token prediction dont mask next entity
         #  this will train the network to predict next token with signals from next entity
         #  for next entity prediction mask next entity to evoid cheating
         # )
        mask_ent = torch.triu(x.new_ones(x_len, seq_len), diagonal=1+m_len).byte()[None,None] if self.mask_ent else None     

        #[None,:,:None] for einsum implementation of attention
        hids = []
        hids_ent = []
        pos = torch.arange(seq_len-1, -1, -1, device=inp.device, dtype=inp.dtype)

        pos_enc = self.pos_enc_wrd(pos)
        pos_enc_ent = self.pos_enc_ent(pos)

        hids.append(inp)
        hids_ent.append(inp_ent)
        for i, layer in enumerate(self.layers):
            # mem = self.hidden[i] if self.mem_len > 0 else None

            inp,inp_ent = layer(inp,inp_ent, r=pos_enc, u=self.u_wrd, v=self.v_wrd, mask=mask, mem=mem[i],r_net=pos_enc_ent, u_ent=self.u_ent,v_ent=self.v_ent,mask_ent=mask_ent,mem_ent=mem_ent[i])
            hids.append(inp)
            hids_ent.append(inp_ent)
        # core_out = inp[:,-x_len:]

        mem =  self._update_mems(mem,hids)
        mem_ent = self._update_mems(mem_ent,hids_ent)
        return mem,mem_ent ,inp,inp_ent

            


In [118]:
model = MaskedKnowledgeAGG(default_wrd_conf,default_ent_conf,512,512,5)

In [119]:
model

MaskedKnowledgeAGG(
  (pos_enc_wrd): Embedding(512, 1024)
  (drop_emb_wrd): Dropout(p=0.0)
  (pos_enc_ent): Embedding(512, 100)
  (drop_emb_ent): Dropout(p=0.0)
  (layers): ModuleList(
    (0): MaskedKnowledgeAGGBlock(
      (mhra_wrd): MultiHeadRelativeAttention(
        (attention): Linear(in_features=1024, out_features=960, bias=False)
        (out): Linear(in_features=320, out_features=1024, bias=False)
        (drop_att): Dropout(p=0.0)
        (drop_res): Dropout(p=0.0)
        (ln): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        (r_attn): Linear(in_features=1024, out_features=320, bias=False)
      )
      (mhra_ent): MultiHeadRelativeAttention(
        (attention): Linear(in_features=100, out_features=960, bias=False)
        (out): Linear(in_features=320, out_features=100, bias=False)
        (drop_att): Dropout(p=0.0)
        (drop_res): Dropout(p=0.0)
        (ln): LayerNorm(torch.Size([100]), eps=1e-05, elementwise_affine=True)
        (r_attn): L